Autorzy:
-Michał Puńko 155863
-Jakub Plura 155829

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def obliczanie_entropii(S):
    wystapienia_klasa = np.bincount(S)
    prawdopodobienstwo = wystapienia_klasa / len(S)
    entropia = 0
    for p in prawdopodobienstwo:
        if p > 0:
            entropia -= p * np.log2(p)
    return entropia

In [3]:
def obliczanie_entropii_warunkowej(S, A):
    wartosci_A, indeksy = np.unique(A, return_inverse=True)
    entropia = 0
    
    for i in range(len(wartosci_A)):
        S_j = np.array(S)[indeksy == i]
        waga = len(S_j) / len(S)
        entropia += waga * obliczanie_entropii(S_j)
    
    return entropia
    

In [4]:
def obliczanie_gain(S,A):
    return obliczanie_entropii(S) - obliczanie_entropii_warunkowej(S,A)

In [5]:
def obliczanie_intrinsic_information(S,A):
    wartosci_A, liczebnosci = np.unique(A, return_counts=True)
    proporcje = liczebnosci / len(S)
    
    wartosc = 0
    for p in proporcje:
        if p > 0:
            wartosc -= p * np.log2(p)
    return wartosc

In [6]:

def obliczanie_gain_ratio(S,A):
    return obliczanie_gain(S,A) / obliczanie_intrinsic_information(S,A)

In [7]:
def formatuj_info_liscia(dane_liscia, class_labels):
    """Pomocnicza funkcja do formatowania etykiety liścia."""
    liczba_przykladow, rozklad_klas = dane_liscia
    info_klas = ", ".join(f"{klasa}: {liczba}" for klasa, liczba in rozklad_klas.items())
    return f"LIŚĆ: [{liczba_przykladow} przykładów, {info_klas}]"

In [8]:
def znajdz_najlepszy_podzial(df, target_name, features):
    """Znajduje cechę z najwyższym Gain Ratio."""
    S = df[target_name].values
    best_feature = None
    max_gain_ratio = -1

    for feature in features:
        A = df[feature].values
        current_gain_ratio = obliczanie_gain_ratio(S, A)
        if current_gain_ratio > max_gain_ratio:
            max_gain_ratio = current_gain_ratio
            best_feature = feature
            
    return best_feature

In [9]:
def buduj_drzewo(df, target_name, features, class_labels):
    """Rekurencyjnie buduje drzewo decyzyjne."""
    
    # --- Warunki stopu (tworzenie liścia) ---
    
    # 1. Jeśli wszystkie przykłady należą do tej samej klasy
    if len(df[target_name].unique()) == 1:
        class_name = class_labels[df[target_name].iloc[0]]
        return (len(df), {class_name: len(df)})

    # 2. Jeśli nie ma już cech do podziału
    if not features:
        majority_class_idx = df[target_name].mode()[0]
        class_name = class_labels[majority_class_idx]
        counts = df[target_name].apply(lambda x: class_labels[x]).value_counts().to_dict()
        return (len(df), counts)

    # --- Krok rekurencyjny (tworzenie węzła) ---
    
    # Znajdź najlepszą cechę do podziału
    best_feature = znajdz_najlepszy_podzial(df, target_name, features)
    
    # Jeśli żadna cecha nie daje przyrostu informacji, stwórz liść
    if best_feature is None:
        counts = df[target_name].apply(lambda x: class_labels[x]).value_counts().to_dict()
        return (len(df), counts)
        
    # Stwórz nowy węzeł drzewa
    tree = {}
    
    # Zaktualizuj listę cech dla kolejnych wywołań
    remaining_features = [f for f in features if f != best_feature]
    
    # Dla każdej unikalnej wartości najlepszej cechy, stwórz gałąź
    for value in df[best_feature].unique():
        sub_df = df[df[best_feature] == value]
        branch_name = f"{best_feature} == {value}"
        
        # Wywołaj rekurencyjnie budowę poddrzewa
        subtree = buduj_drzewo(sub_df, target_name, remaining_features, class_labels)
        tree[branch_name] = subtree
        
    return tree

In [10]:
def rysuj_wezel(wezel, class_labels, prefix=""):
    """Rekurencyjna funkcja do rysowania drzewa."""
    galaz_items = list(wezel.items())
    for i, (warunek, poddrzewo) in enumerate(galaz_items):
        czy_ostatnia = (i == len(galaz_items) - 1)
        lacznik = "└── " if czy_ostatnia else "├── "
        print(prefix + lacznik + warunek)
        
        nowy_prefix = prefix + ("    " if czy_ostatnia else "│   ")
        
        if isinstance(poddrzewo, dict):
            rysuj_wezel(poddrzewo, class_labels, nowy_prefix)
        else:
            info_liscia = formatuj_info_liscia(poddrzewo, class_labels)
            print(nowy_prefix + "└── " + info_liscia)

In [11]:
def wyswietl_drzewo_w_terminalu(drzewo, info_korzenia, class_labels):
    """Główna funkcja, która rozpoczyna rysowanie drzewa."""
    print("--- Drzewo Decyzyjne ---")
    info_startowe = formatuj_info_liscia(info_korzenia, class_labels).replace("LIŚĆ: ", "START")
    print(info_startowe)
    rysuj_wezel(drzewo, class_labels)
    print("------------------------")

In [12]:
df = pd.read_csv("titanic-homework.csv")
df = df.drop(["PassengerId","Name"], axis=1)

In [13]:
przykladowe_S = df["Survived"].values
przykladowe_A = df["Pclass"].values

In [14]:
przykladowe_S = df["Survived"].values
przykladowe_A = df["Pclass"].values
print(f"entropia dla tego x: {obliczanie_entropii(przykladowe_S)}")
print(f"entropia warunkowa dla tych danych: {obliczanie_entropii_warunkowej(przykladowe_S, przykladowe_A)}")
print(f"gain dla tych danych: {obliczanie_gain(przykladowe_S, przykladowe_A)}")
print(f"gain ratio: {obliczanie_gain_ratio(przykladowe_S,przykladowe_A)}")

entropia dla tego x: 0.9709505944546686
entropia warunkowa dla tych danych: 0.8892782366556033
gain dla tych danych: 0.08167235779906523
gain ratio: 0.05960489889898008


In [15]:
kolumna_docelowa = "Survived"
cechy = [col for col in df.columns if col != kolumna_docelowa]

In [16]:
print(f"Budowanie drzewa")
etykiety_klas = {0: 'Nie przeżył', 1: 'Przeżył'}
zbudowane_drzewo = buduj_drzewo(df, kolumna_docelowa, cechy, etykiety_klas)
    
info_korzenia_dict = df[kolumna_docelowa].apply(lambda x: etykiety_klas[x]).value_counts().to_dict()
info_korzenia = (len(df), info_korzenia_dict)
    
wyswietl_drzewo_w_terminalu(zbudowane_drzewo, info_korzenia, etykiety_klas)

Budowanie drzewa
--- Drzewo Decyzyjne ---
START[100 przykładów, Nie przeżył: 60, Przeżył: 40]
├── Sex == male
│   ├── Age == 22
│   │   └── LIŚĆ: [5 przykładów, Nie przeżył: 5]
│   ├── Age == 35
│   │   └── LIŚĆ: [2 przykładów, Nie przeżył: 2]
│   ├── Age == 34
│   │   └── LIŚĆ: [3 przykładów, Nie przeżył: 3]
│   ├── Age == 54
│   │   └── LIŚĆ: [1 przykładów, Nie przeżył: 1]
│   ├── Age == 2
│   │   └── LIŚĆ: [2 przykładów, Nie przeżył: 2]
│   ├── Age == 20
│   │   └── LIŚĆ: [2 przykładów, Nie przeżył: 2]
│   ├── Age == 39
│   │   └── LIŚĆ: [1 przykładów, Nie przeżył: 1]
│   ├── Age == 12
│   │   └── LIŚĆ: [1 przykładów, Przeżył: 1]
│   ├── Age == 28
│   │   ├── Pclass == 1
│   │   │   ├── SibSp == 0
│   │   │   │   └── LIŚĆ: [1 przykładów, Przeżył: 1]
│   │   │   └── SibSp == 1
│   │   │       └── LIŚĆ: [1 przykładów, Nie przeżył: 1]
│   │   └── Pclass == 3
│   │       └── LIŚĆ: [1 przykładów, Nie przeżył: 1]
│   ├── Age == 44
│   │   └── LIŚĆ: [2 przykładów, Nie przeżył: 2]
│   ├── A

/tmp/ipykernel_4824/46879896.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return obliczanie_gain(S,A) / obliczanie_intrinsic_information(S,A)
